# Esse notebook ficará responsável pelo desenvolvimento dos algoritmos de regressão linear para a resolução do problema. Os insights sobre os dados passados vão ser realizados em outro notebook.

In [ ]:
# Realizando todos os imports necessários para a realização do projeto

from google.colab import drive #Importanto o drive do email que você utilizou para se conectar ao colab
drive.mount('/content/drive')  #Estou conectando o meu drive ao colab, já que vou puxar o csv utilizado na resolução do desafio de lá.
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
pd.options.display.float_format = '{:,.0f}'.format  #Para a visualização melhor dos dados de faturamento (não ficar, por exemplo, "e+....")


#O caminho passado aqui deve ser diferente dependendo de como esteja salvo este arquivo no seu drive. Ou seja, é so passar na função "read_csv" o caminho
#do arquivo em csv realizado na resolução do desafio do seu drive. 
desafio = pd.read_csv('/content/drive/My Drive/DesafioEstagio/DesafioEstagioMachineLearning.csv')


Mounted at /content/drive


In [ ]:
# Verificando as correlações entre os dados
corr = desafio.corr()
corr.style.background_gradient(cmap='coolwarm')

,codigo,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
codigo,1.000000,0.058340,0.116533,0.112639,0.106236,0.088400,0.083265,0.075811,0.014112,-0.089280,-0.161552,-0.212292,-0.220039,-0.066785,0.072148,0.158117,0.187358,0.186055,-0.273532,-0.381192
população,0.058340,1.000000,0.970797,0.972057,0.985313,0.990373,0.989377,0.986526,0.991235,0.924673,0.186290,0.293839,0.565726,0.905305,0.928378,0.861343,0.832363,0.822980,-0.026747,-0.166563
popAte9,0.116533,0.970797,1.000000,0.999140,0.994495,0.988292,0.949930,0.934287,0.946176,0.822485,0.043099,0.118759,0.384971,0.790162,0.870195,0.831198,0.808531,0.781555,-0.071316,-0.300307
popDe10a14,0.112639,0.972057,0.999140,1.000000,0.995850,0.987774,0.948990,0.936012,0.949140,0.826978,0.029706,0.108616,0.383258,0.797436,0.876245,0.835387,0.813732,0.784861,-0.078434,-0.306282
popDe15a19,0.106236,0.985313,0.994495,0.995850,1.000000,0.995039,0.970848,0.961319,0.962284,0.852647,0.059684,0.147259,0.425413,0.835775,0.910146,0.869813,0.846498,0.821901,-0.075903,-0.293721
popDe20a24,0.088400,0.990373,0.988292,0.987774,0.995039,1.000000,0.981342,0.967448,0.969580,0.873030,0.103014,0.200898,0.474706,0.856208,0.913433,0.861188,0.833742,0.809816,-0.059987,-0.250416
popDe25a34,0.083265,0.989377,0.949930,0.948990,0.970848,0.981342,1.000000,0.994647,0.969477,0.902734,0.198600,0.303916,0.563934,0.907814,0.951993,0.902184,0.874031,0.863436,-0.037166,-0.195332
popDe35a49,0.075811,0.986526,0.934287,0.936012,0.961319,0.967448,0.994647,1.000000,0.973315,0.916271,0.224111,0.328107,0.588285,0.924404,0.958461,0.905541,0.878573,0.873529,-0.025153,-0.170591
popDe50a59,0.014112,0.991235,0.946176,0.949140,0.962284,0.969580,0.969477,0.973315,1.000000,0.956639,0.247188,0.363377,0.634230,0.924839,0.902424,0.807894,0.774645,0.773025,0.009856,-0.067345
popMaisDe60,-0.089280,0.924673,0.822485,0.826978,0.852647,0.873030,0.902734,0.916271,0.956639,1.000000,0.397839,0.550859,0.802572,0.956781,0.846138,0.715222,0.677137,0.703864,0.074336,0.139008


In [ ]:
# Verificando se há alguma coluna com valores nulos 
desafio.isnull().sum()

codigo          0
nome            0
cidade          0
estado          0
população       0
popAte9         0
popDe10a14      0
popDe15a19      0
popDe20a24      0
popDe25a34      0
popDe35a49      0
popDe50a59      0
popMaisDe60     0
domiciliosA1    0
domiciliosA2    0
domiciliosB1    0
domiciliosB2    0
domiciliosC1    0
domiciliosC2    0
domiciliosD     0
domiciliosE     0
rendaMedia      6
faturamento     0
dtype: int64

In [ ]:
#Verificando quais os indices das linhas que contém o valor nulo para recortalos do dataframe
np.where(desafio.isnull())

(array([  6,  26,  52,  71,  89, 122]), array([21, 21, 21, 21, 21, 21]))

In [ ]:
#Armazenando as linhas com esses valores NaN em outro Dataframe
d1 = desafio.iloc[[6, 26, 52, 71, 89, 122]]
d1

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
6,3304557082,Anil,Rio de Janeiro,RJ,24855,2427,1227,1777,1804,3730,5573,3556,4761,355,759,1771,1863,1902,1155,326,424,nan,"1,092,081"
26,3304557005,Catumbi,Rio de Janeiro,RJ,12910,1773,805,1018,999,1985,2526,1516,2288,0,18,261,640,1281,1140,229,703,nan,"791,170"
52,3304557086,Freguesia (Jacarepaguá),Rio de Janeiro,RJ,72501,7991,3709,5257,5159,10923,16875,9607,12980,698,2569,6042,4941,5055,3919,704,1670,nan,"1,297,388"
71,3304557146,Jacaré,Rio de Janeiro,RJ,9538,1326,629,763,712,1547,1921,1108,1532,0,15,161,492,633,792,197,540,nan,"822,849"
89,3304557032,Maracanã,Rio de Janeiro,RJ,25969,1696,843,1370,1921,4020,5134,4124,6861,739,1695,3191,1924,1522,877,0,497,nan,"1,384,873"
122,3304557006,Rio Comprido,Rio de Janeiro,RJ,44998,5332,2397,3448,3757,7160,9375,5527,8002,0,387,2119,2775,3715,3930,617,1681,nan,"789,463"


In [ ]:
#Retirando as colunas que não iremos utilizar

d1.drop(['codigo', 'nome', 'cidade', 'estado', 'popAte9', 'popDe10a14', 'popDe15a19', 'popDe20a24', 'popDe25a34', 'popDe35a49', 'popDe50a59', 'popMaisDe60'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Reindexando a ordem das colunas
d1 = d1.reindex(columns=['população', 'domiciliosA1', 'domiciliosA2', 'domiciliosB1', 'domiciliosB2', 'domiciliosC1', 'domiciliosC2', 'domiciliosD', 
                    'domiciliosE','faturamento', 'rendaMedia'])

In [ ]:
# Com os valores das linhas que continham valores faltantes armazenados em outro dataframe, já posso retira-las do dataframe inicial
desafio.dropna(inplace=True)
desafio.isnull().sum()

codigo          0
nome            0
cidade          0
estado          0
população       0
popAte9         0
popDe10a14      0
popDe15a19      0
popDe20a24      0
popDe25a34      0
popDe35a49      0
popDe50a59      0
popMaisDe60     0
domiciliosA1    0
domiciliosA2    0
domiciliosB1    0
domiciliosB2    0
domiciliosC1    0
domiciliosC2    0
domiciliosD     0
domiciliosE     0
rendaMedia      0
faturamento     0
dtype: int64

In [ ]:
desafio.head()

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
0,3304557060,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,"2,501","932,515"
1,3304557138,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,"588,833"
2,3304557057,Água Santa,Rio de Janeiro,RJ,9003,883,399,597,762,1755,2076,1112,1419,0,96,404,652,644,522,77,254,"2,391","874,200"
3,3304557031,Alto Da Boa Vista,Rio de Janeiro,RJ,9606,1072,538,660,685,1317,2007,1341,1986,114,178,393,517,945,584,137,286,"3,727","912,226"
4,3304557125,Anchieta,Rio de Janeiro,RJ,57222,7677,3774,4892,4600,8660,12272,7157,8190,0,0,1089,2821,5110,5422,1073,3261,"1,380","553,020"


In [ ]:
#Retirando as colunas que não iremos utilizar do dataframe "desafio" também
desafio.drop(['codigo', 'nome', 'cidade', 'estado', 'popAte9', 'popDe10a14', 'popDe15a19', 'popDe25a34', 'popDe35a49', 'popDe50a59', 
              'popMaisDe60', 'popDe20a24'], axis=1, inplace=True)
desafio.head()

,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
0,11676,0,145,715,1242,1093,758,92,304,"2,501","932,515"
1,27564,0,0,82,506,2040,2490,827,2506,931,"588,833"
2,9003,0,96,404,652,644,522,77,254,"2,391","874,200"
3,9606,114,178,393,517,945,584,137,286,"3,727","912,226"
4,57222,0,0,1089,2821,5110,5422,1073,3261,"1,380","553,020"


In [ ]:
# Dividindo nosso dataset em treino e teste. Como a label que queremos prever é a "rendaMedia", estou adicionando em X todo o resto
X = desafio.drop('rendaMedia', axis=1)

In [ ]:
# Y é a nossa label, ou seja, a "rendaMedia"
y = desafio['rendaMedia']

In [ ]:
# Criamos também a base de dados que realmente queremos prever o valor de "rendaMedia", que continha os valores "NaN"
NovaRendaMedia = d1.drop('rendaMedia', axis=1)

In [ ]:
#Importamos uma biblioteca do sklearn para fazer essa divisão entre as bases de treino e teste

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# No nosso primeiro modelo de regressão, vamos utilizar o tensorflow (que já foi importado lá em cima). Armazenamos na variável "es" os paramêtros de 
# Callback (de parada) que nossa rede neural vai verificar para finalizar o aprendizado durante o processo de fit.
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=30, verbose=0,
    mode='auto')

In [ ]:
#Random seed para reprodutividade
tf.random.set_seed(42)

#Criando o modelo
model_rendaMedia = tf.keras.Sequential([
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dense(1)
])

#Compilando o modelo
model_rendaMedia.compile(loss=tf.keras.losses.mae,
                          optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
                          metrics=['mse'])


#Preenchendo o modelo (fit) #Caso não queria que toda época seja mostrada na tela, colocando o parâmetro ("verbose=0" na função fit resolverá)
model_rendaMedia.fit(X_train, y_train, epochs = 200, callbacks=es)

Epoch 1/200
4/4 [==============================] - 1s 3ms/step - loss: 346054.2812 - mse: 288391102464.0000
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 194760.3906 - mse: 47375704064.0000
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 104070.2891 - mse: 17935773696.0000
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 77511.3125 - mse: 8585255936.0000
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 41472.7422 - mse: 2411399168.0000
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 20939.1680 - mse: 764075456.0000
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 8421.5996 - mse: 105711192.0000
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 6444.9165 - mse: 90091224.0000
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 4847.0400 - mse: 45420472.0000
Epoch 10/200
4/4 [==============================] - 0s 3m

In [ ]:
#Vamos avaliar o quão bem nosso modelo está se saindo (A primeira métrica é a mae (Erro médio absoluto) e a segunda o MSE (Erro quadrático médio))
model_rendaMedia.evaluate(X_test, y_test)

1/1 [==============================] - 0s 110ms/step - loss: 997.1285 - mse: 2299843.5000


[997.1285400390625, 2299843.5]

In [ ]:
# Para compararmos com os outros métodos de regressão mais a frente, vamos armazenar as predições desse modelo em uma variável
predi = model_rendaMedia.predict(X_test)

In [ ]:
#Armazenamos também as predições dele em cima da nossa base de dados que tinha valores nulos
predi_valoresNulos = model_rendaMedia.predict(NovaRendaMedia)

In [ ]:
#Para nosso segundo método de regressão, vamos utilizar o sklearn
from sklearn import linear_model

In [ ]:
#Cria o modelo de regressão
regr = linear_model.LinearRegression()

In [ ]:
# Fazemos o fit do modelo com a base de treino
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Vamos armazenar as predições dele em outra variável para comparações futuras
predi2 = regr.predict(X_test)

In [ ]:
#Nosso terceiro método de regressão também utilizará o SKLEARN, mas dessa vez vamos utilizar o algoritmo de RandomForest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instancia o modelo e faz o fit com os dados de treino
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#Armazena as predições dele em uma variável para comparações futuras
y_pred_test = forest.predict(X_test)

In [ ]:
#Vamos avaliar o quão boas foram nossas predições em cada um dos modelos através da métrica de R2_Score. Basicamente, 1.0 é o valor ótimo
#Ou seja, o score que estiver mais próximo do valor 1 foi o melhor e o mais longe o pior.
from sklearn.metrics import r2_score

In [ ]:
#Parece que as predições feitas pelo algoritmo de regressão lienar do sklearn não foram muito legais
r2_score(y_test, predi2)

-0.5938557627876921

In [ ]:
#As predições do nosso modelo de rede neural estão bem próximos do valor 1. Ótimo.
r2_score(y_test, predi)

0.8314335112993541

In [ ]:
# As predições feitas pelo nosso algoritmo de RandomForest também foram bem próximas.
r2_score(y_test, y_pred_test)

0.7484215349972776

In [ ]:
#Como as predições do nosso algoritmo de rede neural foi a que mais se aproximou de 1.0, é ele que usaremos para prever os valores NaN
predi_valoresNulos = tf.squeeze(predi_valoresNulos) #O modelo nos retornou um tensor de dimensão (2,1), então usamos a função squeeze para retirar 
                                                    #a dimensão unitária
Valores_Nulos = predi_valoresNulos.numpy() #Transformando o tensor em um array numpy para assim adicionarmos os valores no dataframe

In [ ]:
#Verificamos se ele foi realmente convertido em um array
Valores_Nulos

array([3577.1682, 2151.3894, 3959.3384, 2702.7622, 7062.405 , 2091.494 ],
      dtype=float32)

In [ ]:
#Adicionamos os valores previstos no dataframe no lugar dos valores "NaN"
d1['rendaMedia'] = Valores_Nulos

In [ ]:
#Verificando como estão dispostos nossos dois dataframes

d1

,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,faturamento,rendaMedia
6,24855,355,759,1771,1863,1902,1155,326,424,"1,092,081","3,577"
26,12910,0,18,261,640,1281,1140,229,703,"791,170","2,151"
52,72501,698,2569,6042,4941,5055,3919,704,1670,"1,297,388","3,959"
71,9538,0,15,161,492,633,792,197,540,"822,849","2,703"
89,25969,739,1695,3191,1924,1522,877,0,497,"1,384,873","7,062"
122,44998,0,387,2119,2775,3715,3930,617,1681,"789,463","2,091"


In [ ]:
desafio

,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
0,11676,0,145,715,1242,1093,758,92,304,"2,501","932,515"
1,27564,0,0,82,506,2040,2490,827,2506,931,"588,833"
2,9003,0,96,404,652,644,522,77,254,"2,391","874,200"
3,9606,114,178,393,517,945,584,137,286,"3,727","912,226"
4,57222,0,0,1089,2821,5110,5422,1073,3261,"1,380","553,020"
...,...,...,...,...,...,...,...,...,...,...,...
155,18791,0,259,774,1363,1923,1379,190,405,"2,316","901,446"
156,13556,0,219,714,828,1051,1012,116,465,"2,416","849,731"
157,33190,0,906,2866,2241,2982,1200,360,772,"3,189","1,127,042"
158,8867,0,87,556,836,940,484,75,214,"2,418","921,875"


In [ ]:
#Reindexando d1 para que ele fique igual nosso dataframe desafio

d1 = d1.reindex(columns=['população', 'domiciliosA1', 'domiciliosA2', 'domiciliosB1', 'domiciliosB2', 'domiciliosC1', 'domiciliosC2', 'domiciliosD', 
                    'domiciliosE','rendaMedia', 'faturamento'])
d1

,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
6,24855,355,759,1771,1863,1902,1155,326,424,"3,577","1,092,081"
26,12910,0,18,261,640,1281,1140,229,703,"2,151","791,170"
52,72501,698,2569,6042,4941,5055,3919,704,1670,"3,959","1,297,388"
71,9538,0,15,161,492,633,792,197,540,"2,703","822,849"
89,25969,739,1695,3191,1924,1522,877,0,497,"7,062","1,384,873"
122,44998,0,387,2119,2775,3715,3930,617,1681,"2,091","789,463"


In [ ]:
#Concatenando os dois dataframes para reincerir as linhas que anteriormente continham os valores NaN (Os valores foram adicionados nas ultimas 6 linhas)
desafio_concat = pd.concat([desafio, d1], ignore_index=True)
desafio_concat

,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
0,11676,0,145,715,1242,1093,758,92,304,"2,501","932,515"
1,27564,0,0,82,506,2040,2490,827,2506,931,"588,833"
2,9003,0,96,404,652,644,522,77,254,"2,391","874,200"
3,9606,114,178,393,517,945,584,137,286,"3,727","912,226"
4,57222,0,0,1089,2821,5110,5422,1073,3261,"1,380","553,020"
...,...,...,...,...,...,...,...,...,...,...,...
155,12910,0,18,261,640,1281,1140,229,703,"2,151","791,170"
156,72501,698,2569,6042,4941,5055,3919,704,1670,"3,959","1,297,388"
157,9538,0,15,161,492,633,792,197,540,"2,703","822,849"
158,25969,739,1695,3191,1924,1522,877,0,497,"7,062","1,384,873"


In [ ]:
# Verificando as correlações entre os dados novamente, agora sem os valores NaN e com menos colunas
corr = desafio.corr()
corr.style.background_gradient(cmap='coolwarm')

,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
população,1.000000,0.185050,0.292196,0.565363,0.905439,0.928215,0.861135,0.832294,0.823415,-0.026747,-0.169300
domiciliosA1,0.185050,1.000000,0.929321,0.719123,0.255843,0.046048,-0.041649,-0.072007,0.063286,0.502224,0.757360
domiciliosA2,0.292196,0.929321,1.000000,0.896174,0.438076,0.168001,0.034866,-0.000580,0.114435,0.414954,0.781901
domiciliosB1,0.565363,0.719123,0.896174,1.000000,0.744524,0.476200,0.302998,0.260777,0.329335,0.245900,0.611988
domiciliosB2,0.905439,0.255843,0.438076,0.744524,1.000000,0.921113,0.799239,0.762202,0.755079,-0.021212,0.009096
domiciliosC1,0.928215,0.046048,0.168001,0.476200,0.921113,1.000000,0.961086,0.940559,0.903801,-0.128446,-0.327108
domiciliosC2,0.861135,-0.041649,0.034866,0.302998,0.799239,0.961086,1.000000,0.988650,0.956927,-0.166935,-0.481882
domiciliosD,0.832294,-0.072007,-0.000580,0.260777,0.762202,0.940559,0.988650,1.000000,0.955385,-0.177658,-0.515005
domiciliosE,0.823415,0.063286,0.114435,0.329335,0.755079,0.903801,0.956927,0.955385,1.000000,-0.138307,-0.414675
rendaMedia,-0.026747,0.502224,0.414954,0.245900,-0.021212,-0.128446,-0.166935,-0.177658,-0.138307,1.000000,0.556335


### Agora, vamos partir para o problema de regressão caso se escolha expandir para outros bairros

In [ ]:
# Dividindo nosso dataset em treino e teste. Como a label que queremos prever agora é "faturamento", estou adicionando em X todo o resto
X1 = desafio_concat.drop('faturamento', axis=1)

In [ ]:
# Y é a nossa label, ou seja, o "faturamento"
y2 = desafio_concat['faturamento']

In [ ]:
#Importamos uma biblioteca do sklearn para fazer essa divisão entre as bases de treino e teste

from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X1, y2, test_size=0.2, random_state=42)

In [ ]:
# A viriável e paramêtros do EarlyStopping que vamos usar são os mesmos. Eu trouxe para cá para não termos que modificar lá em cima
#Toda vez que quisermos modificar algum parâmetro
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=30, verbose=0,
    mode='auto')

In [ ]:
#Random seed para reprodutividade
tf.random.set_seed(42)

#Criando o modelo
model_Faturamento = tf.keras.Sequential([
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1)
])

#Compilando o modelo
model_Faturamento.compile(loss=tf.keras.losses.mae,
                          optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
                          metrics=['mse'])


#Preenchendo o modelo (fit) #Caso não queria que toda época seja mostrada na tela, colocando o parâmetro ("verbose=0" na função fit resolverá)
model_Faturamento.fit(X_train2, y_train2, epochs = 600, callbacks=es)

Epoch 1/600
4/4 [==============================] - 0s 3ms/step - loss: 854334.3750 - mse: 933473550336.0000
Epoch 2/600
4/4 [==============================] - 0s 3ms/step - loss: 805568.7500 - mse: 800382255104.0000
Epoch 3/600
4/4 [==============================] - 0s 3ms/step - loss: 708864.0000 - mse: 1042377211904.0000
Epoch 4/600
4/4 [==============================] - 0s 3ms/step - loss: 667159.5000 - mse: 1766020087808.0000
Epoch 5/600
4/4 [==============================] - 0s 3ms/step - loss: 632257.7500 - mse: 936676818944.0000
Epoch 6/600
4/4 [==============================] - 0s 3ms/step - loss: 581392.2500 - mse: 877606928384.0000
Epoch 7/600
4/4 [==============================] - 0s 3ms/step - loss: 556122.4375 - mse: 925504176128.0000
Epoch 8/600
4/4 [==============================] - 0s 7ms/step - loss: 518183.9062 - mse: 550203162624.0000
Epoch 9/600
4/4 [==============================] - 0s 4ms/step - loss: 508480.8750 - mse: 486742622208.0000
Epoch 10/600
4/4 [========

In [ ]:
#Validando a performance do nosso algoritmo. Como se observa, ele não se saiu muito bem.
model_Faturamento.evaluate(X_test2, y_test2)

1/1 [==============================] - 0s 83ms/step - loss: 137887.4688 - mse: 37750210560.0000


[137887.46875, 37750210560.0]

In [ ]:
#Armazenando as predições dele em uma variável para testá-las com o R2 Score.
predi_F = model_Faturamento.predict(X_test2)

In [ ]:
# Como mostra no resultado, ele foi um pouco abaixo do esperado. Vamos tentar utilizar os algoritmos do sklearn.
r2_score(y_test2, predi_F)

0.6468970227553663

In [ ]:
# Utilizando o algoritmo de regressão linear do sklearn.
regr.fit(X_train2, y_train2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Vamos armazenar as predições dele também em uma variável para verificar o quão bem se saiu
predi_F2 = regr.predict(X_test2)

In [ ]:
# Bem melhor do que o nosso algoritmo de rede neural. Esse é um bom sinal.
r2_score(y_test2,predi_F2)

0.9168379468087178

In [ ]:
# Vamos fazer a mesma coisa, só que agora utilizando um algoritmo de Random Forest.
forest.fit(X_train2, y_train2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Armazenando as predições do nosso algoritmo.
predi_F3 = forest.predict(X_test2)

In [ ]:
# Ele se saiu um pouco melhor do que nosso algoritmo de Regressão Linear. 
r2_score(y_test2,predi_F3)

0.9030367376565912

In [ ]:
#Essa é a função que vai retonar o possível faturamento em um novo bairro. Ela utiliza as predições feitas pelo nosso algoritmo
# de RandomForest para gerar as predições. Essa função recebe um dataframe que informe: A população, a quantidade de cada tipo de domicilio
# naquele bairro e a rendaMedia, e retorna esse mesmo dataframe com uma nova coluna chamada "Faturamento", que seria uma estimativa do faturamento
# naquele bairro. 

def NovoBairro (dataframe=X_test2):  #O valor padrão de "dataframe é o X_test2" só para ficar de forma ilustrativa. Em uma chamada real, se chamaria
                                     #NovoBairro(dataframe="o nome do dataframe com as informações necessárias")
  predicoes = forest.predict(X_test2)
  X_test2['Faturamento'] = predicoes

  return X_test2

In [ ]:
# Chamada da função que preve o faturamento. Nesse exemplo, utilizo o dataframe que foi separado para ser utilizado como teste para nosso
# Algoritmo de regressão linear. Mas, em um caso real, iriamos ter outro dataframe e teriamos que inseri-lo como parâmetro. 
NovoBairro()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,população,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,Faturamento
105,12088,0,93,330,564,1076,1049,185,858,"1,688","806,674"
108,65958,0,484,3359,4409,6273,4442,780,2418,"2,162","860,621"
141,28018,31,160,693,911,2588,2895,497,1154,"1,740","684,686"
55,9935,95,326,1295,1221,1218,427,39,187,"4,011","959,473"
94,3270,0,10,115,244,380,329,65,108,"1,696","837,355"
29,1442,0,3,45,146,159,64,8,23,"1,934","919,451"
101,80900,0,239,2129,4629,7639,6086,1082,3563,"1,682","629,794"
51,13480,0,0,91,480,1334,1253,240,1025,"1,133","711,591"
100,9666,0,0,231,662,876,783,150,532,"1,522","808,624"
142,15908,0,32,467,846,1612,1474,302,935,"1,550","764,380"


In [ ]:
## Salvando os modelos
model_rendaMedia.save("Modelo_RendaMedia.h5")

In [ ]:
from joblib import dump, load
dump(forest, 'NovoBairro.joblib') 

['NovoBairro.joblib']